# Transformers 模型量化技术：AWQ（OPT-2.7B）

![img](https://huggingface.co/datasets/ybelkada/documentation-images/resolve/main/Thumbnail.png)

在2023年6月，Ji Lin等人发表了论文 [AWQ：Activation-aware Weight Quantization for LLM Compression and Acceleration](https://arxiv.org/pdf/2306.00978.pdf)。

这篇论文详细介绍了一种激活感知权重量化算法，可以用于压缩任何基于 Transformer 的语言模型，同时只有微小的性能下降。关于 AWQ 算法的详细介绍，见[MIT Han Song 教授分享](https://hanlab.mit.edu/projects/awq)。

transformers 现在支持两个不同的 AWQ 开源实现库：

- [AutoAWQ](https://github.com/casper-hansen/AutoAWQ)
- [LLM-AWQ](https://github.com/mit-han-lab/llm-awq) 


因为 LLM-AWQ 不支持 Nvidia T4 GPU（课程演示 GPU），所以我们使用 AutoAWQ 库来介绍和演示 AWQ 模型量化技术。

## 使用 AutoAWQ 量化模型

下面我们以 `facebook opt-2.7B` 模型为例，使用 `AutoAWQ` 库实现的 AWQ 算法实现模型量化。

In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

#model_name_or_path = "/home/intel/models/qwen2-7B-instruct"
#model_name_or_path = "/home/intel/.cache/huggingface/hub/models--THUDM--chatglm3-6b/snapshots/91a0561caa089280e94bf26a9fc3530482f0fe60"
model_name_or_path = "/home/intel/models/Baichuan2-7B-Chat"
quant_model_dir = "models/Baichuan2-awq"


quant_config = {
    "zero_point": True,
    "q_group_size": 128,
    "w_bit": 4,
    "version": "GEMM"
}

/home/intel/miniforge3/envs/peft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 加载模型
model = AutoAWQForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [3]:

# 量化模型
model.quantize(tokenizer, quant_config=quant_config)

Repo card metadata block was not found. Setting CardData to empty.
Token indices sequence length is longer than the specified maximum sequence length for this model (8815 > 4096). Running this sequence through the model will result in indexing errors
AWQ:   0%|          | 0/32 [00:00<?, ?it/s]/home/intel/miniforge3/envs/peft/lib/python3.10/site-packages/torch/backends/cuda/__init__.py:342: FutureWarning: torch.backends.cuda.sdp_kernel() is deprecated. In the future, this context manager will be removed. Please see, torch.nn.attention.sdpa_kernel() for the new context manager, with updated signature.
  warnings.warn(
AWQ: 100%|██████████| 32/32 [08:54<00:00, 16.70s/it]


### 实测 AWQ 量化模型：GPU显存占用峰值超过10GB


```shell
Sun Dec 24 15:21:35 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   53C    P0              71W /  70W |   7261MiB / 15360MiB |     97%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|```

In [4]:
quant_config

{'zero_point': True, 'q_group_size': 128, 'w_bit': 4, 'version': 'GEMM'}

#### Transformers 兼容性配置

为了使`quant_config` 与 transformers 兼容，我们需要修改配置文件：`使用 Transformers.AwqConfig 来实例化量化模型配置`

In [23]:
from transformers import AwqConfig, AutoConfig

# 修改配置文件以使其与transformers集成兼容
quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

# 预训练的transformers模型存储在model属性中，我们需要传递一个字典
model.model.config.quantization_config = quantization_config
print(quantization_config)

{'quant_method': <QuantizationMethod.AWQ: 'awq'>, 'bits': 4, 'group_size': 128, 'zero_point': True, 'version': <AWQLinearVersion.GEMM: 'gemm'>, 'backend': <AwqBackendPackingMethod.AUTOAWQ: 'autoawq'>, 'fuse_max_seq_len': None, 'modules_to_not_convert': None, 'modules_to_fuse': None, 'do_fuse': False, 'load_in_4bit': True}


In [24]:
# 保存模型权重
model.save_quantized(quant_model_dir)
# 保存分词器
tokenizer.save_pretrained(quant_model_dir)  

('models/Baichuan2-awq/tokenizer_config.json',
 'models/Baichuan2-awq/special_tokens_map.json',
 'models/Baichuan2-awq/tokenizer.model',
 'models/Baichuan2-awq/added_tokens.json')

In [25]:
model.eval()

BaichuanAWQForCausalLM(
  (model): BaichuanForCausalLM(
    (model): BaichuanModel(
      (embed_tokens): Embedding(125696, 4096, padding_idx=0)
      (layers): ModuleList(
        (0-31): 32 x DecoderLayer(
          (self_attn): Attention(
            (W_pack): WQLinear_GEMM(in_features=4096, out_features=12288, bias=False, w_bit=4, group_size=128)
            (o_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=False, w_bit=4, group_size=128)
            (rotary_emb): RotaryEmbedding()
          )
          (mlp): MLP(
            (gate_proj): WQLinear_GEMM(in_features=4096, out_features=11008, bias=False, w_bit=4, group_size=128)
            (down_proj): WQLinear_GEMM(in_features=11008, out_features=4096, bias=False, w_bit=4, group_size=128)
            (up_proj): WQLinear_GEMM(in_features=4096, out_features=11008, bias=False, w_bit=4, group_size=128)
            (act_fn): SiLU()
          )
          (input_layernorm): RMSNorm()
          (post_attention_layernorm): R

### 使用 GPU 加载量化模型

In [37]:
from transformers import AutoTokenizer, AutoModelForCausalLM


# 加载预训练模型配置
config = AutoConfig.from_pretrained(quant_model_dir,trust_remote_code=True)
config.quantization_config = quantization_config

tokenizer = AutoTokenizer.from_pretrained(quant_model_dir,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(quant_model_dir,trust_remote_code=True,device_map="cuda").to(0)

KeyError: 'load_in_4bit'

In [9]:
def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to(0)

    out = model.generate(**inputs, max_new_tokens=64)
    return tokenizer.decode(out[0], skip_special_tokens=True)


In [ ]:
result = generate_text("Merry Christmas! I'm glad to")
print(result)

In [ ]:
result = generate_text("The woman worked as a")
print(result)